## Conversational Summarization

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

### T5 MODEL FOR ABSTRACTIVE SUMMARIZATION
Abstractive summarizers are so-called because they do not select sentences from the originally given text passage to create the summary. Instead, they produce a paraphrasing of the main contents of the given text, using a vocabulary set different from the original document. This is very similar to what we as humans do, to summarize. 

### Gist of the things that were tried-

#### 1] Summarization per response for a given question asked to a parent 
Observation - Didn't work out well when output's max/min length weren't defined
#### 2] Summarization per response for a given question asked to a parent without the question test pre-pended 
Observation - Misleading summaries
#### 3] Summarization per response for a given question asked to a parent with the question test pre-pended
Observation - T5 model was pre-trained on CNN dataset and CNN references were being added
#### 4] Building up on the above scenario and also adding moderator/parent words to the question/answer text. 

Ex: Moderator 1 : "How was the use of technology before the pandemic?"
Parent 4: "It was very limited. We only.. "

Observation - Passing data in this format resulted in better summaries than the rest above. 
Also setting min,max_length was useful. 
But still not good enough as sentences are getting repeated in the summary.

In [3]:
#loading pre-trained model
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

In [4]:
#loading pre-trained embeddings
tokenizer = AutoTokenizer.from_pretrained("t5-base")

In [6]:
input_text = """Oh okay. Well, they didn't use it as much. 
I mean, the teachers would assign some reading or math assignments on I ready and from time to time some homework.
But yeah, that was about it. I would monitor how much time they were on Roblox and other things to entertain themselves,
but it was easier control them back then because they didn't have to go on as much. I mean, that's what it was."""

In [7]:
def get_summary(input_text):
    input_text_tokens = tokenizer("summarize: "+ input_text, return_tensors="pt", max_length=512, truncation=True )
    output_summary = model.generate(input_text_tokens["input_ids"], max_length=250, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(output_summary[0])

In [8]:
print(get_summary(input_text))

<pad> the teachers would assign some reading or math assignments on I ready and from time to time some homework. but yeah, that was about it. back then, it was easier control them because they didn't have to go on as much.</s>


In [9]:
input_text = """I would say the same thing.
My daughter was in fifth grade, we would have a couple in Newsela projects, half an hour of homework.
But we were in the playground, neighborhood friends, after school projects, after school swimming.
A little bit of tablet on the weekends, but there was no internet access and there was no YouTube in my life, there was no Roblox, there was no gaming. 
So, a very different world.
"""

In [10]:
print(get_summary(input_text))
#repition observed - there was no Roblox

<pad> there was no internet access, there was no Roblox, there was no gaming. my daughter was in fifth grade, we would have a couple in Newsela projects, half an hour of homework. there was no YouTube in my life, there was no Roblox.</s>


In [23]:
input_text = """I can say for my eight-year-old, because again, we went from not really knowing how to use a computer
or how to type, to transition really smoothly, I mean if you can call that a benefit, we transitioned really quickly. 
He was typing away, quickly was able to navigate where to go to sign into a Google classroom. 
So, that was impressive in the beginning, how quickly they were able to adapt to that. 
So yes... I'm sorry, can you repeat the question?"""

In [24]:
print(get_summary(input_text))
# unnecessary text at the end of the summary

<pad> eight-year-old transitioned from not knowing how to use a computer to using a google classroom. he was typing away, quickly was able to navigate where to go to sign into a google classroom. so yes... can you repeat the question?</s>


In [16]:
input_text = """Moderator 1: "If you can walk me through a little bit how your use of technology changed through the pandemic
when they had to start using it more for school or what your situation was for your child?""
Parent 1: "I can say for my eight-year-old, because again, we went from not really knowing how to use a computer or 
how to type, to transition really smoothly, I mean if you can call that a benefit, we transitioned really quickly. 
He was typing away, quickly was able to navigate where to go to sign into a Google classroom. 
So, that was impressive in the beginning, how quickly they were able to adapt to that. 
So yes... I'm sorry, can you repeat the question?""
"""

In [17]:
print(get_summary(input_text))

<pad> "i mean if you can call that a benefit, we transitioned really smoothly," says a parent. "he was typing away, quickly was able to navigate where to go to sign into a google classroom"</s>


In [18]:
input_text = """If you can walk me through a little bit how your use of technology changed through the pandemic when they had to start using it more for school or what your situation was for your child?
I can say for my eight-year-old, because aga"in, we went from not really knowing how to use a computer or how to type, to transition really smoothly, I mean if you can call that a benefit, we transitioned really quickly. He was typing away, quickly was able to navigate where to go to sign into a Google classroom. So, that was impressive in the beginning, how quickly they were able to adapt to that. So yes... I'm sorry, can you repeat the question?
"""

In [19]:
print(get_summary(input_text))
#unnecessary new words added which are not part of the original text

<pad> aga"in, 8, was able to transition from not knowing how to use a computer to a google classroom. aga'in was typing away, quickly was able to navigate where to go to sign into a google classroom.</s>


In [25]:
input_text = """Moderator 1: "So just starting to think through some of the impact of technology, can we talk a little bit about how your child used technology prior to the pandemic for school purposes?
"
Parent 1: "Oh okay. Well, they didn't use it as much. 
I mean, the teachers would assign some reading or math assignments on I ready and from time to time some homework. 
But yeah, that was about it. 
I would monitor how much time they were on Roblox and other things to entertain themselves, 
but it was easier control them back then because they didn't have to go on as much. I mean, that's what it was.
"
"""

In [26]:
print(get_summary(input_text))

<pad> parents can talk about how their children used technology prior to the pandemic. "i would monitor how much time they were on Roblox and other things to entertain themselves" "it was easier control them back then because they didn't have to go on as much"</s>


In [39]:
input_text = """Moderator 1: "So just starting to think through some of the impact of technology, can we talk a little bit about how your child used technology prior to the pandemic for school purposes?"
Parent 2: "Oh okay. Well, they didn't use it as much. I mean, the teachers would assign some reading or math assignments on I ready and from time to time some homework. But yeah, that was about it. I would monitor how much time they were on Roblox and other things to entertain themselves, but it was easier control them back then because they didn't have to go on as much. I mean, that's what it was."
Parent 3: "I would say the same thing. My daughter was in fifth grade, we would have a couple in Newsela projects, half an hour of homework. But we were in the playground, neighborhood friends, after school projects, after school swimming. A little bit of tablet on the weekends, but there was no internet access and there was no YouTube in my life, there was no Roblox, there was no gaming. So, a very different world."
Parent 5: "I have a similar experience to Parent 3. My high schooler had a lot of his homework that he had to type and word process... I don't know if that's the right word, into Docs, but it wasn't a huge amount. It wasn't doing research, it wasn't hours and hours. My elementary school guy, he didn't have any homework on the computer at all. And my middle schooler had very little, like type this one thing or something like that. It was very little on the computer. And for myself, in terms of my family, I lock the video games so that there was very little access to video games during the week."
Parent 5: "And then on the weekends there was an hour or something like that, it wasn't that much. There was no YouTube watching, like now YouTube consumes our life. My kids didn't even know how to use the features on our TV, the smart features, they didn't know they could access YouTube on it or anything like that. So it was very controlled and very little. I also had my children in a lot of afterschool activities. My older son was in an orchestra, he also was in a band, theater groups, there was a lot of afterschool activity. A lot of playing outside, a lot of going to the park and all that stuff. Yeah, so..."
Parent 4: "I can go. My kids probably had more... It sounds like they did more online than, so far, the rest of them. But my daughter in high school definitely had almost all of her homework... Not all of it because she was assigned in school, but she definitely was on doing homework, I feel like, a good amount. Nothing like now, but definitely was using it. My middle schooler also, but not as much. They both did play games or use Instagram, my daughter was really into Instagram, not as much anymore but definitely games. Now it's just like a free-for-all. I mean, they're in their rooms with the door shut, I have no idea what they're doing all day. I just know that they're doing their homework, getting good grades. I think they're on electronics the entire day."
Parent 1: "Yeah, so I have a similar experience with my eight-year-old. Pre-pandemic he was still in second grade so the use of computers was very limited. Maybe once a week he would type, words of the week was one of the options I picked to do. That was his only interaction with computer. Of course he had devices to play on, and he would go to his father's house every other weekend and he would be more exposed there to computers. But still he was busy. He was busy during the week, there was school, there was getting to school, getting from school, that's a trip in itself."
Parent 1: "There was hockey, we had TaeKwonDo, he was very busy. Even then it didn't seem like enough, it didn't seem like we were doing enough but he was certainly a lot busier than now. And he did get to play, mostly probably on the weekends, and again, weekends were also pretty packed. So I don't know how he had time, but he did play, not a lot. And of course we were able to control it, it was an item you had to earn and if you didn't do well then you don't get to play. So it was a tool we could use as incentive during the week. So now that's all out the window, it's all available right there all day long. And getting him away from the computer now is a project in itself every day."
"""

In [40]:
print(get_summary(input_text))
#captured content is good but a lot of information is missed out

<pad> parents say their children used technology for school purposes prior to the pandemic. "it was easier control them because they didn't have to go on as much," one parent says. "my kids didn't even know how to use the features on our TV, the smart features"</s>


In [27]:
input_text = """Moderator 1: "How the technology has changed during the pandemic, school purposes and whether it's beneficial or problematic?"
Parent 1: "Right, exactly, it seems beneficial because it is an important part of your life as you grow older in high school clearly, and in middle school as Parent 4 said, there's some use of computer. So at first it seemed like a good thing, and I'm sure it is a good thing, it's just, like I said, it's just become overwhelming use of technology because they sit in front of it all day. And another thing, this use of YouTube, because I understand teachers post some things that you can look up on YouTube, so you can't block YouTube because it'll be used as a tool for one of those classes they have."
Parent 1: "But YouTube is just this bottomless hole of... I can't even begin, it's just so bad. And transition from looking at your Google classroom or your Google meet and here's YouTube, it's just so easy. And we were not always there, we still have jobs and other things to do, can't just sit on top of them. So this accessibility of technology that has this type of content, like YouTube, that's not beneficial for kids, that's been problematic, to say the least."
Parent 5: "So for us, we only had a desktop and a laptop, which was more than sufficient pre-pandemic. And then everybody needed a device, all of a sudden everybody needed a device Needed to be on at basically the same times so we got one device from school, we bought a laptop, we found an old laptop. It was like the gathering of devices. And then teaching... My nine-year-old, teaching him how to use it, he really hadn't used computers in that way. And so teaching him how to use it. My middle son is prone to migraines and so being on the screen caused migraines and so that caused some aversion because he didn't want to be on."
Parent 5: "The schedules were kind of crazy, like all over the place. And the truth is that it's hard because my 17-year-old, who knew how to do all these things just kind of stayed in his room for months. And it was just... I feel bad saying it, but he was kind of left to his own devices. He wasn't a big social media guy, he has no social media still, but when the only way to communicate with his peers became to use... What is it? Discord, and all these other things, that became the only way to communicate with people. And then it almost, I feel like... Like Parent 1 said, it's like this black hole that sucks you in."
Parent 5: "And then there was also no schedule. Because we have some special needs in our family, my schedule is really tight with my kids and so the little guys we go to bed at 8:30 and then 9:00 and then the older one was in bed by 10:00 and there was a lot of structure, and literally it was like a bomb hit it. People were going to bed at 1:00 in the morning, it was just totally... It felt chaotic and out of control, and it felt like the technology sputtered for a little bit as we gt devices figured out how to get everybody on and then it was this black hole that sucked you in. And then everybody was on for hours and hours and hours and hours and hours, and it felt out of control."
Parent 3: "Sure, when lock down happened my then first grader, the school did a great job pivoting to remote quite quickly. They used seesaw as a platform. I did get two laptops from the DOE because we have a lot of tech but it's all very old and kind of fragile and I'm like, they're going to wreck it all. Nothings insured. We're self-employed and I could just see the whole business being wiped out so we did take a DOE iPads which was a lifesaver. But also, again, we got to one point where the computer teacher did that YouTube... We were not on the YouTube and she sent one link and they figured out, "Oh, I can get to YouTube from this link.""
Parent 3: "And then it was like a bomb went off, they figured it out, they figured out workarounds and it was like Pandora's box. I was trying to figure out how to block... I was on the phone with the DOE, I was on the phone with a parent coordinator, I was calling the principal, like, "How can I block YouTube without..." I was putting timer things, it was driving me crazy, and it was downhill from there. I finally figured out my son, by banging on it so much he's actually broken the screen so whatever controls I put in, he could bang on it and it would just do whatever he wanted. A month later I exchange the iPad but I nearly ripped my own hair out over this YouTube. Yeah, the addiction sunk in over those first five months of lockdown of screens and YouTube and just that misery sucking."
Parent 4: "Like I said, both my kids are on it all day. They're fully remote, both of them. Neither one of them has a full schedule so I hear my son... Because my desk is right next to his room, and I'm hearing him at like 10:00 in the morning, profanities on a game with his friend. And I'm like, I don't understand, it's a school day. Well, they don't have school, it's in between a break. So it's definitely made our day... Especially since I'm working right next to him, very difficult to try and do my job and be next to him. But it's their lives, they're in their rooms, if it's not school then it's playing games. My son plays a lot of games with his friends, my daughter doesn't but then she'll be on the YouTube also a lot, TickTock, whatever, and I kind of don't see them a lot during the day. Thankfully they do okay so I don't ask... I used to be a more involved and now I'm not really involved. Now it's a free for all."
Parent 4: "Yeah, I don't feel like there's even much to be involved. They don't have much, especially my middle Schooler, has nothing to do so there isn't even much for me to ask him."
"""

In [28]:
print(get_summary(input_text))
#text duplication

<pad> "youtube is just this bottomless hole of... I can't even begin, it's just so bad," says one parent. "my 17-year-old, who knew how to do all these things just kind of stayed in his room for months" "it's hard because my 17-year-old, who knew how to do all these things just kind of stayed in his room"</s>


### Conclusion -
Abstractive summarization results using pre-trained model was not good enough. And since training would have meant building summaries, as a next step Extractive summarization was tried